In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
S, P = np.load("../data/dataset.npy")
molecules = np.load("../data/molecules.npy")

In [4]:
from SCFInitialGuess.utilities.dataset import Dataset

dim = 26

ind_cut = 150
index = np.arange(200)
np.random.shuffle(index)

S_test = np.array(S)[index[150:]]
P_test = np.array(P)[index[150:]]
molecules_test = [molecules[index[i]] for i in range(150, 200)]

S_train = np.array(S)[index[:150]]
P_train = np.array(P)[index[:150]]
molecules_train = [molecules[index[i]] for i in range(150)]

dataset = Dataset(np.array(S_train), np.array(P_train), split_test=0.0)

dataset.testing = (Dataset.normalize(S_test, mean=dataset.x_mean, std=dataset.x_std)[0], P_test)

[-] 2018-03-26 12:02:33: Data set normalized. Mean value std: 0.008754558889917387


In [7]:
from SCFInitialGuess.nn.networks import EluTrNNN
from SCFInitialGuess.nn.training import Trainer, RegularizedMSE

trainer = Trainer(
    EluTrNNN([dim**2, 600, 400, 200, 100, dim**2]),
    cost_function=RegularizedMSE(alpha=1e-7),
    optimizer=tf.train.AdamOptimizer(learning_rate=1e-3)
)

trainer.setup()
network_orig, sess_orig = trainer.train(
    dataset,
    convergence_threshold=1e-7
)
graph_orig = trainer.graph

[-] 2018-03-26 12:05:06: No target graph specified for Trainer setup. Creating new graph ...
[-] 2018-03-26 12:05:06: Setting up the training in the target graph ...
[-] 2018-03-26 12:05:06: network ...
[-] 2018-03-26 12:05:06: error function ...
[-] 2018-03-26 12:05:06: cost function ...
[-] 2018-03-26 12:05:06: training step
[-] 2018-03-26 12:05:07: Starting network training ...
[ ] 2018-03-26 12:05:07: Val. Cost: 5.776E-02. Error: 5.776E-02. Diff: 1.0E+10
[ ] 2018-03-26 12:05:09: Val. Cost: 2.540E-04. Error: 2.497E-04. Diff: 5.8E-02
[ ] 2018-03-26 12:05:12: Val. Cost: 1.026E-04. Error: 9.814E-05. Diff: 1.5E-04
[ ] 2018-03-26 12:05:14: Val. Cost: 4.950E-05. Error: 4.488E-05. Diff: 5.3E-05
[ ] 2018-03-26 12:05:16: Val. Cost: 3.163E-05. Error: 2.697E-05. Diff: 1.8E-05
[ ] 2018-03-26 12:05:19: Val. Cost: 1.324E-05. Error: 8.560E-06. Diff: 1.8E-05
[ ] 2018-03-26 12:05:21: Val. Cost: 1.068E-05. Error: 6.082E-06. Diff: 2.5E-06
[ ] 2018-03-26 12:05:24: Val. Cost: 1.040E-05. Error: 5.896E-06

$P'= \dfrac{3}{2} PSP - \dfrac{2}{2 \cdot 2}PSPSP $

In [8]:
def mc_wheeny_purification(p,s):
    p = p.reshape(dim, dim)
    s = s.reshape(dim, dim)
    return (3 * np.dot(np.dot(p, s), p) - np.dot(np.dot(np.dot(np.dot(p, s), p), s), p)) / 2

def multi_mc_wheeny(p, s, n_max=4):
    for i in range(n_max):
        p = mc_wheeny_purification(p, s)
    return p

def idemp_error(p, s):
    p = p.reshape(dim, dim)
    s = s.reshape(dim, dim)
    return np.mean(np.abs(np.dot(np.dot(p, s), p) - 2 * p))


In [9]:
#for (s, p) in zip(*dataset.testing):
for (s, p) in zip(S_test, P_test):
    
    #s_norm = s.reshape(1, dim**2)
    s_norm = dataset.input_transformation(s.reshape(1, dim**2))
    
    print("Orip:         {:0.3E}".format(idemp_error(p, s))) 
    print("Orig prurif:  {:0.3E}".format(idemp_error(mc_wheeny_purification(p, s), s)))
    
    with graph_orig.as_default():
        p_nn = network_orig.run(sess_orig, s_norm).reshape(dim, dim)
        
    print("NN:           {:0.3E}".format(idemp_error(p_nn, s)))
    print("NN pruified:  {:0.3E}".format(idemp_error(mc_wheeny_purification(p_nn, s), s)))
    p_nn_multi = multi_mc_wheeny(p_nn, s, n_max=5)
    print("NN multified: {:0.3E}".format(idemp_error(p_nn_multi, s))) 
    print("Value before: {:0.3E}".format(np.mean(np.abs(p.reshape(dim, dim) - p_nn))))
    print("Value multi :  {:0.3E}".format(np.mean(np.abs(p.reshape(dim, dim) - p_nn_multi))))
    print("Is nan: " + str(np.sum(np.isnan(p_nn_multi))))
    print("Is inf: " + str(np.sum(np.isinf(p_nn_multi))))
    print("Is fin: " + str(np.sum(np.isfinite(p_nn_multi))))
    print("--------------------")

Orip:         3.068E-16
Orig prurif:  8.853E-17
NN:           1.989E-03
NN pruified:  4.080E-05
NN multified: 6.369E-17
Value before: 1.357E-03
Value after:  8.945E-04
Is nan: 0
Is inf: 0
Is fin: 676
--------------------
Orip:         3.191E-16
Orig prurif:  8.043E-17
NN:           1.571E-03
NN pruified:  2.384E-05
NN multified: 6.539E-17
Value before: 1.205E-03
Value after:  9.513E-04
Is nan: 0
Is inf: 0
Is fin: 676
--------------------
Orip:         3.424E-16
Orig prurif:  8.940E-17
NN:           2.157E-03
NN pruified:  5.321E-05
NN multified: 6.865E-17
Value before: 1.415E-03
Value after:  9.595E-04
Is nan: 0
Is inf: 0
Is fin: 676
--------------------
Orip:         3.416E-16
Orig prurif:  9.039E-17
NN:           2.103E-03
NN pruified:  4.238E-05
NN multified: 7.526E-17
Value before: 1.521E-03
Value after:  1.059E-03
Is nan: 0
Is inf: 0
Is fin: 676
--------------------
Orip:         2.625E-16
Orig prurif:  8.785E-17
NN:           1.561E-03
NN pruified:  1.988E-05
NN multified: 6.593E

Value after:  9.890E-04
Is nan: 0
Is inf: 0
Is fin: 676
--------------------
Orip:         3.625E-16
Orig prurif:  9.193E-17
NN:           1.787E-03
NN pruified:  3.318E-05
NN multified: 7.202E-17
Value before: 1.212E-03
Value after:  8.647E-04
Is nan: 0
Is inf: 0
Is fin: 676
--------------------
Orip:         2.924E-16
Orig prurif:  8.689E-17
NN:           2.041E-03
NN pruified:  4.826E-05
NN multified: 7.906E-17
Value before: 1.363E-03
Value after:  8.723E-04
Is nan: 0
Is inf: 0
Is fin: 676
--------------------
Orip:         3.175E-16
Orig prurif:  8.878E-17
NN:           1.969E-03
NN pruified:  4.420E-05
NN multified: 7.622E-17
Value before: 1.287E-03
Value after:  8.444E-04
Is nan: 0
Is inf: 0
Is fin: 676
--------------------
Orip:         3.151E-16
Orig prurif:  7.743E-17
NN:           2.287E-03
NN pruified:  7.093E-05
NN multified: 6.673E-17
Value before: 1.711E-03
Value after:  1.233E-03
Is nan: 0
Is inf: 0
Is fin: 676
--------------------


In [10]:
from pyscf.scf import hf
from SCFInitialGuess.utilities.plotutils import prediction_scatter

dim = 26



iterations = []
for i, (molecule, p) in enumerate(zip(molecules_test, P_test)):
    
    mol = molecule.get_pyscf_molecule()
    
    
    print("Calculating: " + str(i + 1) + "/" + str(len(molecules_test)))
    

    s_raw = hf.get_ovlp(mol)
    s_norm = dataset.input_transformation(s_raw.reshape(1, dim**2))
    
    with graph_orig.as_default():
        P_orig = network_orig.run(sess_orig, s_norm).reshape(dim, dim).astype('float64')
        
        P_orig_sym = (P_orig + P_orig.T) / 2
        P_orig_idem = multi_mc_wheeny(P_orig, s_raw, n_max=5)
        
        # check errors
        print("Accuracy (MSE):")
        print(" -Orig: {:0.3E}".format(np.mean(np.abs(p.reshape(dim, dim) - P_orig)**2)))
        print(" -Sym:  {:0.3E}".format(np.mean(np.abs(p.reshape(dim, dim) - P_orig_sym)**2)))
        print(" -Idem: {:0.3E}".format(np.mean(np.abs(p.reshape(dim, dim) - P_orig_idem)**2)))
        
        print("Idempotency:")
        print(" -Orig: {:0.3E}".format(idemp_error(P_orig, s_raw)))
        print(" -Sym:  {:0.3E}".format(idemp_error(P_orig_sym, s_raw)))
        print(" -Idem: {:0.3E}".format(idemp_error(P_orig_idem, s_raw)))

    
    # P_actual wi noise
    p_noise = p.reshape(dim, dim) + np.random.randn(dim, dim) * 1e-4
    
    
    iterations_molecule = []

    for guess in [p_noise, P_orig, P_orig_sym, P_orig_idem]:
        mf = hf.RHF(mol)
        mf.diis = None
        mf.verbose = 1
        mf.kernel(dm0=guess)
        iterations_molecule.append(mf.iterations)
    
    iterations.append(iterations_molecule)

iterations = np.array(iterations)

Warn: Ipython shell catchs sys.args


Calculating: 1/50
Accuracy (MSE):
Orig: 3.325E-06
Sym:  2.881E-06
Idem: 1.421E-06
Idempotency:
Orig: 1.989E-03
Sym:  1.818E-03
Idem: 6.369E-17


Warn: Ipython shell catchs sys.args


Calculating: 2/50
Accuracy (MSE):
Orig: 2.904E-06
Sym:  2.595E-06
Idem: 1.839E-06
Idempotency:
Orig: 1.571E-03
Sym:  1.436E-03
Idem: 6.539E-17


Warn: Ipython shell catchs sys.args


...

Calculating: 48/50
Accuracy (MSE):
Orig: 3.379E-06
Sym:  2.918E-06
Idem: 1.332E-06
Idempotency:
Orig: 2.041E-03
Sym:  1.863E-03
Idem: 7.906E-17


Warn: Ipython shell catchs sys.args


Calculating: 49/50
Accuracy (MSE):
Orig: 3.126E-06
Sym:  2.660E-06
Idem: 1.240E-06
Idempotency:
Orig: 1.969E-03
Sym:  1.791E-03
Idem: 7.622E-17


Warn: Ipython shell catchs sys.args


Calculating: 50/50
Accuracy (MSE):
Orig: 6.043E-06
Sym:  5.652E-06
Idem: 3.129E-06
Idempotency:
Orig: 2.287E-03
Sym:  2.133E-03
Idem: 6.673E-17


In [11]:
print(np.mean(iterations,0))

[  5.62  12.06  12.04   9.12]
